In [44]:
def f1_score_at_k(label, prediction, k):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    # convert label and prediction strings to sets of key-phrases
    label_set = set([key_phrase.strip() for key_phrase in label.split(';')])
    print(f'label_set: {label_set}')
    
    # split the predicted key-phrases and their scores
    prediction_set = set([key_phrase.strip() for key_phrase in prediction.split(';')[:k]])
    # prediction_set = set(p[0] for p in predictions[:k])
    print(f'prediction_set: {prediction_set}')
    
    # calculate true positives, false positives, and false negatives
    for keyphrase in prediction_set:
        if keyphrase in label_set:
            true_positives += 1
        else:
            false_positives += 1
    
    for keyphrase in label_set:
        if keyphrase not in prediction_set:
            false_negatives += 1
    
    print(f'true_positives: {true_positives}')    
    print(f'false_positives: {false_positives}')
    print(f'false_negatives: {false_negatives}')

    # calculate precision, recall, and F1 score
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    
    if precision == 0 or recall == 0:
        return 0
    
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    return f1_score

In [45]:
label = '김민재; 나폴리; 챔피언스리그; 이탈리아 클럽; AC 밀란; 인터 밀란; 8강 진출; 선발 출전; 후반 추가골; 피오트르 지엘린스키'
prediction = '홀란드; 캐피탈; 분데스리가; 이탈리아; AC 밀란; 인터 밀란; 8강 진출; 선발 출전; 후반 추가골; 피오트르 지엘린스키; 크앙크앙'

In [48]:
f1_score_at_k(label, prediction, 12)

label_set: {'이탈리아 클럽', '8강 진출', '챔피언스리그', '인터 밀란', '후반 추가골', 'AC 밀란', '김민재', '선발 출전', '피오트르 지엘린스키', '나폴리'}
prediction_set: {'분데스리가', '8강 진출', '홀란드', '크앙크앙', '캐피탈', '이탈리아', '인터 밀란', '후반 추가골', 'AC 밀란', '선발 출전', '피오트르 지엘린스키'}
true_positives: 6
false_positives: 5
false_negatives: 4


0.5714285714285713

In [49]:
f1_score_at_k(label, prediction, 10)

label_set: {'이탈리아 클럽', '8강 진출', '챔피언스리그', '인터 밀란', '후반 추가골', 'AC 밀란', '김민재', '선발 출전', '피오트르 지엘린스키', '나폴리'}
prediction_set: {'분데스리가', '8강 진출', '홀란드', '캐피탈', '이탈리아', '인터 밀란', '후반 추가골', 'AC 밀란', '선발 출전', '피오트르 지엘린스키'}
true_positives: 6
false_positives: 4
false_negatives: 4


0.6